数字化转型 对 营运绩效 的影响
数字化转型、双元创新与营运绩效


In [50]:
# 中介变量：行业类别/企业规模/股权性质/企业年限/股权集中度

# ['总资产周转率', '固定资产周转率', '非流动资产周转率', '流动资产周转率', '应付账款周转率', '应收账款周转率', '营运资金(资本)周转率', '股东权益周转率', '现金及现金等价物周转率', '存货周转率']

# 企业营运能力
# 固定资产周转率/应收账款周转率/非流动资产周转/应付账款周转率/总资产周转率/存货周转率

# 研发投入，创新产出
# 适配能力(Fit) = 创新投入(11)/创新产出(I0)


# 吴非称之为 适配能力
# 1.企业创新效率(EFF)
# 效率评价的常用方法有两种:随机前沿分析法(SFA)和数据包络分析法(DEA)。通常而言,数据包络分析法(DEA)会因离散程度较大而导致效率值被低估。鉴于此,本文采用研发费用的自然对数作为衡量创新投人的指标,采用专利授权数的自然对数作为衡量创新产出的指标
# 利用随机前沿分析法(SFA)估算创新效率。此外,为了增强结论的稳健性,将利用数据包络分析法(DEA)估算的创新效率作为稳健性检验。————数字化转型对制造业企业创新效率的门槛效应研究

In [ ]:
inditor_dict = {
    '盈利能力': ['资产报酬率', '流动资产净利润率', '固定资产净利润率', '总资产净利润率', '净资产收益率', '投资收益率',
                 '投入资本回报率', '营业净利率', '成本费用利润率', '营业利润率', '销售期间费用率', '托宾Q值',
                 '总营业成本率'],
    '营运能力': ['总资产周转率', '固定资产周转率', '非流动资产周转率', '流动资产周转率', '应付账款周转率',
                 '应收账款周转率', '营运资金(资本)周转率', '股东权益周转率', '现金及现金等价物周转率', '存货周转率'],
    '债偿能力': ['流动比率', '现金比率', '速动比率', '资产负债率', '产权比率', '权益乘数', '负债与权益市价比率',
                 '现金资产比率', '经营活动产生的现金流量净额/负债合计'],
    '发展能力': ['净利润增长率', '利润总额增长率', '净资产收益率增长率', '固定资产增长率', '总资产增长率', '资本积累率',
                 '资本保值增值率', '销售费用增长率', '管理费用增长率', '营业总收入增长率', '营业总成本增长率',
                 '基本每股收益增长率', '营业利润增长率', '经营活动产生的净流量增长率', '投资活动产生的现金流量增长率',
                 '营业费用增长率'],
    '现金能力':['净利润现金净含量', '营业利润现金净含量', '营业收入现金净含量', '全部现金回收率', '营运指数', '资本支出与折旧摊销比'],
    '风险能力': ['财务杠杆', '经营杠杆'],
    '社会贡献力': ['员工收入增长率', '提供岗位增长率'],
    '创新能力': ['研发人员数量占比', '创新产出', '研发投入'],
    '其他': ['股权集中度', '供应链集中', '董事会规模', '两权分离度'],
}

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

path = r'C:\onedrivers\OneDrive - jsnu.edu.cn\projects\digital\04.企业绩效评价研究\datas\数字化差异性分析数据.csv'
dataset = pd.read_csv(path, dtype={'股票代码':'object'})
dataset_ = pd.read_csv(r'C:\datas\企业信息数据\上市企业补充信息数据.csv', dtype={'股票代码':'object'})
dataset_.drop(['上市省份','上市城市'],axis=1,inplace=True)
dataset = pd.merge(dataset,dataset_, how='left',on=['股票代码', '截止日期'])
dataset['上市状态'] = dataset['上市状态'].fillna('ST')
dataset = dataset[~dataset['上市状态'].str.contains('ST')]
def entropy(matrix):
    """熵权法求取权重"""
    names = matrix.columns
    project, indicator = matrix.shape
    for name in names:
        max_ = matrix[name].max()
        min_ = matrix[name].min()
        matrix[name] = (matrix[name] - min_) / (max_ - min_)
    # 计算熵权权重
    p = pd.DataFrame(np.zeros([project, indicator], dtype='float64'), columns=names)
    for i in range(indicator):
        p.iloc[:, i] = matrix.iloc[:, i] / matrix.iloc[:, i].sum()
    E = -1 / np.log(project) * (p * np.log(p)).sum()
    return (1 - E) / sum((1 - E))
# names = ['股票代码', '截止日期', '研发投入', '创新产出', '企业规模', '股权集中度', '企业年限', '行业类别', '股权性质', '行业代码', '数字化转型程度(李寿喜)']
digital = ['数字化转型程度(赵宸宇)', '数字化转型程度(文本)', '数字化转型程度(量化)']
min_ = 1.098612
dataset['创新产出'] = dataset['创新产出'].apply(lambda x: min_ if x==0 else x)
dataset['技术创新A'] = (dataset['研发投入']+dataset['创新产出'])/2
dataset['创新效率A'] = dataset['研发投入']/dataset['创新产出']

operational = ['固定资产周转率', '应收账款周转率', '非流动资产周转率', '应付账款周转率', '总资产周转率', '存货周转率']
performence = ['托宾Q值', '净利润增长率', '经营活动产生的净流量增长率', '资本积累率', '利润总额增长率', '固定资产净利润率', '总资产净利润率', '投入资本回报率']
matrix = dataset[operational]
dataset['营运绩效'] = matrix.dot(entropy(matrix))
matrix = dataset[performence[1:]]
dataset['盈利绩效'] = matrix.dot(entropy(matrix))
matrix = dataset[digital]
dataset['digital'] =  matrix.dot(entropy(matrix))
dataset['截止日期'] = dataset['截止日期'].astype(int)
needs = ['股票代码','截止日期','行业代码','资产负债率','盈利绩效','托宾Q值','营运绩效','技术创新','股权性质','企业年限','企业规模','股权集中度','digital','数字化转型程度(量化)','数字化转型程度(文本)'] # ,'数字化转型程度(赵宸宇)'

data = dataset[needs] # 股票代码
data = data[(data['行业代码']!=39)&(data['截止日期']>2013)].set_index(['行业代码','截止日期'])

data.rename(columns={'数字化转型程度(文本)':'digitalt','数字化转型程度(量化)':'digitalc'}, inplace=True)
data

股票代码     资产负债率      盈利绩效      托宾Q值      营运绩效      技术创新  股权性质  \
行业代码 截止日期                                                                   
37   2015  000008  0.837709  0.774854  0.492356  0.028443  0.649479   0.0   
     2016  000008  0.767586  0.490377  0.373824  0.021760  0.651378   1.0   
     2017  000008  0.668323  0.378681  0.329522  0.027311  0.704525   1.0   
     2018  000008  0.710860  0.318309  0.158982  0.024318  0.724637   1.0   
     2019  000008  0.613962  0.323329  0.120871  0.024599  0.690475   1.0   
...           ...       ...       ...       ...       ...       ...   ...   
27   2022  688366  0.894736  0.235332  0.131359  0.017672  0.695776   0.0   
     2019  688399  0.929230  0.558402  0.083217  0.032282  0.823555   0.0   
     2020  688399  0.739636  0.533975  0.348335  0.106980  0.501386   0.0   
     2021  688399  0.732285  0.409737  0.142780  0.068521  0.477285   0.0   
     2022  688399  0.791119  0.406564  0.054243  0.087149  0.449742   0.0   

           企业年限       企业规模     股权集中度   digital  digitalc  digitalt  
行业代码 截止日期                                                           
37   2015  23.0  22.001238  0.543948  0.077841  0.032254  0.012588  
     2016  24.0  22.826504  0.259287  0.109414  0.024791  0.019890  
     2017  25.0  23.091481  0.302993  0.174253  0.024418  0.035458  
     2018  26.0  23.071433  0.447653  0.286120  0.102571  0.036286  
     2019  27.0  23.211878  0.369529  0.326798  0.207630  0.032465  
...         ...        ...       ...       ...       ...       ...  
27   2022   3.0  22.653685  0.855962  0.015464  0.000000  0.005171  
     2019   0.0  20.866814  0.548727  0.042642  0.000000  0.012786  
     2020   1.0  21.546420  0.515946  0.033081  0.000000  0.010679  
     2021   2.0  21.975841  0.391151  0.062926  0.000000  0.014443  
     2022   3.0  22.310663  0.366476  0.075337  0.000000  0.015832  

[10644 rows x 13 columns]

In [4]:
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

# 建立模型
data_ = data.copy()
control = ['资产负债率', '股权性质', '企业年限', '企业规模', '股权集中度']

# data = data.set_index(["firm","year"])
# 数字化转型——>技术创新——>营运能力

# needs = ['股票代码','截止日期','行业代码',
# '资产负债率','股权性质','企业年限','企业规模','股权集中度',
# '盈利能力', '托宾Q值','营运能力'
# '适配能力',
# 'digital','数字化转型程度(量化)','数字化转型程度(文本)']
# + 股权集中度

model = PanelOLS.from_formula("营运绩效 ~ digital + 股权性质 + 股权集中度+ 企业规模  + 企业年限 + EntityEffects + TimeEffects", data=data_)
model.fit()

Dep. Variable:,营运绩效,R-squared:,0.0196
Estimator:,PanelOLS,R-squared (Between):,0.6175
No. Observations:,10644,R-squared (Within):,0.0199
Date:,"Thu, Mar 28 2024",R-squared (Overall):,0.3215
Time:,14:50:28,Log-likelihood,1.41e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,42.286
Entities:,28,P-value,0.0000
Avg Obs:,380.14,Distribution:,"F(5,10603)"
Min Obs:,24.000,,
Max Obs:,1327.0,F-statistic (robust):,42.286


In [5]:
# 
model = PanelOLS.from_formula("技术创新 ~ digital + 股权性质 + 股权集中度+ 企业规模  + 企业年限 + EntityEffects + TimeEffects", data=data_)
model.fit()

Dep. Variable:,技术创新,R-squared:,0.0647
Estimator:,PanelOLS,R-squared (Between):,-3.3086
No. Observations:,10644,R-squared (Within):,0.0557
Date:,"Thu, Mar 28 2024",R-squared (Overall):,-2.6556
Time:,14:50:58,Log-likelihood,5148.9
Cov. Estimator:,Unadjusted,,
,,F-statistic:,146.78
Entities:,28,P-value,0.0000
Avg Obs:,380.14,Distribution:,"F(5,10603)"
Min Obs:,24.000,,
Max Obs:,1327.0,F-statistic (robust):,146.78


In [6]:
# 
model = PanelOLS.from_formula("营运绩效 ~ digital + 技术创新 + 股权性质 + 股权集中度+ 企业规模  + 企业年限 + EntityEffects + TimeEffects", data=data_)
model.fit()

Dep. Variable:,营运绩效,R-squared:,0.0371
Estimator:,PanelOLS,R-squared (Between):,-0.3812
No. Observations:,10644,R-squared (Within):,0.0365
Date:,"Thu, Mar 28 2024",R-squared (Overall):,-0.2577
Time:,14:51:14,Log-likelihood,1.419e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,68.021
Entities:,28,P-value,0.0000
Avg Obs:,380.14,Distribution:,"F(6,10602)"
Min Obs:,24.000,,
Max Obs:,1327.0,F-statistic (robust):,68.021


In [66]:
model = PanelOLS.from_formula("技术创新 ~ digital + 股权性质 + 股权集中度 + 企业规模 + 企业年限 + EntityEffects + TimeEffects", data=data_)
model.fit()

Dep. Variable:,技术创新,R-squared:,0.0257
Estimator:,PanelOLS,R-squared (Between):,0.7592
No. Observations:,18282,R-squared (Within):,0.0342
Date:,"Wed, Mar 27 2024",R-squared (Overall):,0.6896
Time:,15:07:03,Log-likelihood,8582.8
Cov. Estimator:,Unadjusted,,
,,F-statistic:,96.301
Entities:,30,P-value,0.0000
Avg Obs:,609.40,Distribution:,"F(5,18232)"
Min Obs:,2.0000,,
Max Obs:,2505.0,F-statistic (robust):,96.301


In [32]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
# data_ = data[(data['行业类别']==36)] # &(data['截止日期']==2020)
# data_ = data.copy()
data_ = data_.set_index(['股票代码','截止日期'])
# data_.describe()
data_.columns=['营运能力','适配能力','digital','企业规模','行业类别','股权集中度','股权性质','企业年限']

model = smf.ols(formula='营运能力~ 适配能力 + digital + 企业规模 + 股权集中度 + 股权性质 + 企业年限', data=data_)  # + C(行业类别)

# data_['intercept']=1 
# X = data_[['intercept','适配能力','digital','企业规模','行业类别','股权性质','股权集中度','企业年限']]  # 
# y = data_['营运能力']
# model = sm.OLS(y,X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   营运能力   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     48.17
Date:                Wed, 27 Mar 2024   Prob (F-statistic):           1.12e-47
Time:                        11:52:35   Log-Likelihood:                -1492.2
No. Observations:                2468   AIC:                             2996.
Df Residuals:                    2462   BIC:                             3031.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0021      0.000     15.097      0.0